In [3]:
import keras
import pandas as pd
import json
import textdistance
import re
from collections import Counter
import numpy as np

### Load model và các file cần thiết cho quá trình dự đoán

In [5]:
model = keras.models.load_model("./main_model3.h5")
raw_data= pd.read_csv('./main_data.csv', index_col=[0])
stats = pd.read_csv('./main_stats.csv', index_col=[0])
with open('./model_features.json', 'r', encoding='utf8') as f:
  model_features = json.load(f)
with open('./catogorical_features.json', 'r', encoding='utf8') as f:
  categorical_features = json.load(f)

In [6]:
keys = ['TinhTrang',  'Hang',  'DongXe',  'SoKmDaDi',  'NamSanXuat',  'DiaDiem',  'SoCho',  'XuatXu',  'KieuDang',  'NhienLieu',  'HopSo']

In [7]:
def real(x):
  return (x * stats.loc['GiaTien', 'std']) + stats.loc['GiaTien','mean']

In [8]:
num_features = len(model_features)

### Hàm sửa lỗi chính tả trong trường hợp người dùng nhập sai chính tả

In [9]:
def my_autocorrect(input_word, V, word_freq_dict, probs):
    input_word = input_word.lower()
 
    if input_word in V:
        return input_word
    else:
        similarities = [1-(textdistance.Jaccard(qval=2).distance(v,input_word)) for v in word_freq_dict.keys()]
        df = pd.DataFrame.from_dict(probs, orient='index').reset_index()
        df = df.rename(columns={'index':'Word', 0:'Prob'})
        df['Similarity'] = similarities
        output = df.sort_values(['Similarity', 'Prob'], ascending=False).head().reset_index()
        return(output.loc[0, 'Word'])    
def correct(field, input_word):
  words = raw_data[field]
 
  # This is our vocabulary
  V = categorical_features[field]

  word_freq_dict = {}  
  word_freq_dict = Counter(words)  
  
  # print(word_freq_dict.most_common()[0:10])
  probs = {} 
      
  Total = sum(word_freq_dict.values())
      
  for k in word_freq_dict.keys():
      probs[k] = word_freq_dict[k]/Total

  return my_autocorrect(input_word, V, word_freq_dict, probs)

### Hàm dự đoán giá xe, đầu vào là các thuộc tính đã được định nghĩa sẵn, lần lượt là: tình trạng, hãng xe, dòng xe, số km đã đi, năm sản xuất, địa điểm, số chỗ, xuất xứ, kiểu dáng, nhiêu liệu, hộp số.

In [12]:
def predict(values):
  new_values = []
  for field, value in zip(keys, values):
    if field != 'SoKmDaDi' and field != 'SoCho' and field != 'NamSanXuat':
      new_values.append(correct(field, value))
    else:
      new_values.append(value)
  res = dict(zip(keys, new_values))
  x = np.zeros(num_features,)
  for key in res:
    # key = unicodedata.normalize('NFC', key)
    if key == 'SoKmDaDi':
      idx = model_features[key]
      x[idx] = res[key]
    elif key == 'NamSanXuat':
      idx = model_features['Tuoi']
      tuoi = 2020 - res[key]
      x[idx] = tuoi
    elif key == 'SoCho':
      idx = model_features['SoCho']
      x[idx] = res[key]
    else:
      idx = model_features['_'.join([key, res[key]])]
      x[idx] = 1
  x = (x - np.array((list(stats['mean'])[1:])))/np.array((list(stats['std'])[1:]))
  x = x.reshape(1, num_features)
  y = model.predict(x)[0][0]
  return int(real(y))

### Dự đoán

In [13]:
values = ['mới', 'Toyota', 'Innova', 0, 2020, 'Hà Nội', 4, 'Trong nước', 'Minivan (MPV)', 'Xăng', 'Số sàn']
predict(values)

585255392

### Nhập dữ liệu từ bàn phím

In [14]:
values = []
for i in keys:
  print('Nhap {}: '.format(i))
  if i == 'SoKmDaDi' or i == 'SoCho':
    x = int(input())
    while x < 0:
      print('Yêu cầu nhập lại:')
      x = int(input())  
    values.append(x)
  elif i == 'NamSanXuat':
    x = int(input())
    while x < 0 or x > 2020:
      print('Yêu cầu nhập lại:')
      x = int(input())
    values.append(x) 
  else:
    values.append(input())

Nhap TinhTrang: 
Nhap Hang: 
Nhap DongXe: 
Nhap SoKmDaDi: 
Nhap NamSanXuat: 
Nhap DiaDiem: 
Nhap SoCho: 
Nhap XuatXu: 
Nhap KieuDang: 
Nhap NhienLieu: 
Nhap HopSo: 


In [17]:
new_values = []
for field, value in zip(keys, values):
  if field != 'SoKmDaDi' and field != 'SoCho' and field != 'NamSanXuat':
    new_values.append(correct(field, value))
  else:
    new_values.append(value)
print('Đây là thông tin về xe của bạn:')
print(dict(zip(keys, new_values)))

Đây là thông tin về xe của bạn:
{'TinhTrang': 'Mới', 'Hang': 'toyota', 'DongXe': 'camry', 'SoKmDaDi': 0, 'NamSanXuat': 2020, 'DiaDiem': 'Hà Nội', 'SoCho': 4, 'XuatXu': 'Nhập khẩu', 'KieuDang': 'Sedan', 'NhienLieu': 'Xăng', 'HopSo': 'Tự động'}


In [18]:
print('Y/c dữ đoán: ấn 1, thoát ấn 0')
x = int(input())
if x == 1:
  print(predict(new_values))
else:
  print('good bye')

Y/c dữ đoán: ấn 1, thoát ấn 0
883274559
